# Set environment

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Oct 17 16:18:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pandas as pd
import numpy as np
import glob
import sys
import os
import re

In [ ]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, KFold
from sklearn import svm

In [ ]:
!pip install datasets transformers==4.9.2 seqeval
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 25.2 MB/s 
     |████████████████████████████████| 2.6 MB 41.1 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 880 kB 56.1 MB/s 
     |████████████████████████████████| 3.3 MB 40.7 MB/s 
     |████████████████████████████████| 441 kB 56.1 MB/s 
     |████████████████████████████████| 432 kB 57.3 MB/s 
     |████████████████████████████████| 431 kB 57.2 MB/s 
     |████████████████████████████████| 431 kB 46.8 MB/s 
     |████████████████████████████████| 365 kB 59.1 MB/s 
     |████████████████████████████████| 362 kB 51.5 MB/s 
     |████████████████████████████████| 362 kB 58.2 MB/s 
     |████████████████████████████████| 361 kB 57.8 MB/s 
     |████████████████████████████████| 346 kB 56.2 MB/s 
     |████████████████████████████████| 86 kB 3.7 MB/s 
     |█████████████████████████████

In [ ]:
from transformers import (
    AutoModel, AutoTokenizer, AutoConfig,
    AutoModelForTokenClassification,
    TrainingArguments, Trainer,
    DataCollatorForTokenClassification
)

from datasets import (
    load_dataset, load_metric,
    Dataset,
    DatasetDict,
    Features, Sequence, ClassLabel, Value
)
from sklearn.metrics import classification_report

In [ ]:
lm_name = 'airesearch/wangchanberta-base-att-spm-uncased'
model = AutoModel.from_pretrained(lm_name)
tokenizer = AutoTokenizer.from_pretrained(lm_name)

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/423M [00:00<?, ?B/s]

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertModel: ['lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/282 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/905k [00:00<?, ?B/s]

# Loading the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/Colab_Notebooks/IS/Test_IS

/content/drive/MyDrive/Colab_Notebooks/IS/Test_IS


In [ ]:
!pwd

/content/drive/MyDrive/Colab_Notebooks/IS/Test_IS


In [ ]:
train_df = pd.read_csv('train_df_normalized.csv', encoding='utf-8-sig')
dev_df = pd.read_csv('dev_df_normalized.csv', encoding='utf-8-sig')
test_df = pd.read_csv('test_df_normalized.csv', encoding='utf-8-sig')

In [ ]:
df_pre = pd.concat([train_df, dev_df, test_df]).reset_index(drop=True)

In [ ]:
df_pre.shape

(300, 7)

In [ ]:
df_pre['tokens'] = df_pre['tokens'].apply(lambda x: x.split('|'))
df_pre['num_label'] = df_pre['num_label'].apply(lambda x: x.split('|'))

In [ ]:
df = df_pre.drop(columns=['text', 'MASK_NUM', 'PRON', 'REF', 'n_tokens'])

In [ ]:
df

,tokens,num_label
0,"[ปีเตอร์, นะ, , ฉัน, ได้ยิน, ข่าว, ว่า, จอห์น...","[N, N, N, N, N, N, N, N, N, N, mask_3rd, N, N, N]"
1,"[ทำไม, ส่วนใหญ่, ญาติ, ต้อง, อิจฉา, กันเอง, ,...","[N, N, N, N, N, N, N, N, mask_3rd, N, N, N, N]"
2,"[เมื่อก่อน, ตอน, เธอ, อ้วน, , mask, ดู, แก่, ...","[N, N, N, N, N, mask_2nd, N, N, N, N, N]"
3,"[ไป, ถาม, พี่, เขา, หน่อย, ซิ, ว่า, mask, อยาก...","[N, N, N, N, N, N, N, mask_3rd, N, N, N, N]"
4,"[เฮีย, แก, กลับมา, ร้องเพลง, ได้, แล้ว, , สงส...","[N, N, N, N, N, N, N, N, mask_3rd, N, N, N, N]"
...,...,...
295,"[เมื่อวาน, mask, ได้, เก็บ, หนังสือ, ไว้, ให้,...","[N, mask_2nd, N, N, N, N, N, N, N, N, N, N, N, N]"
296,"[mask, ไม่ต้อง, ทำ, ถึงขนาด, นั้น, ก็ได้, นะ, ...","[mask_2nd, N, N, N, N, N, N, N, N, N, N, N, N,..."
297,"[mask, ต้อง, ขอโทษ, ด้วย, ค่ะ, , น้อง, ธุรการ...","[mask_1st, N, N, N, N, N, N, N, N, N, N, N, N]"
298,"[ถ้า, คราวนี้, แก, ไม่, ยอม, ทำ, ตามที่, ฉัน, ...","[N, N, N, N, N, N, N, N, N, N, N, mask_2nd, N,..."


# Functions

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    tokens = [tokenizer.convert_ids_to_tokens(x) for x in tokenized_inputs["input_ids"] ]
    ids=[]
    labels = []
    for i, label in enumerate(examples["num_label"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # Set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)
        ids.append(word_ids)

    tokenized_inputs["labels"] = labels
    tokenized_inputs["word_ids"] = ids
    tokenized_inputs["tokens_bert"] = tokens
    return tokenized_inputs


In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
def flatten(t):
    return [num_label[item] for sublist in t for item in sublist]

# KFold

In [ ]:
# shuffle ไม่ให้เรียง label
df = df.sample(frac=1, random_state=33).reset_index(drop=True)
df.head()

,tokens,num_label
0,"[มึง, , กู, ว่า, พี่, เอ้, หล่อ, , แม้ว่า, m...","[N, N, N, N, N, N, N, N, N, mask_3rd, N, N, N]"
1,"[กู, ไป, ไม่, ได้, นะ, , พรุ่งนี้, mask, ไป, ...","[N, N, N, N, N, N, N, mask_1st, N, N, N, N]"
2,"[mask, จะ, ใส่, ชุด, อะไร, ไป, ก็ได้, , ไม่, ...","[mask_2nd, N, N, N, N, N, N, N, N, N, N, N, N]"
3,"[มึง, ช่วย, ให้คำปรึกษา, กู, เรื่อง, คุม, น้ำห...","[N, N, N, N, N, N, N, N, N, N, mask_1st, N, N,..."
4,"[ป้า, ทำความสะอาด, ห้อง, ให้, เสร็จ, แล้ว, ค่ะ...","[N, N, N, N, N, N, N, N, N, mask_1st, N, N, N,..."


In [ ]:
#เก็บ index ของ mask
mask_index = df['tokens'].apply(lambda x: x.index('mask'))

In [ ]:
list_predict = []
list_actual = []
list_input_test = []

kf = KFold(n_splits=3, shuffle=True, random_state=45)

for train_index, test_index in kf.split(df):
  #train_x = df.loc[train_index, 'tokens']
  #train_y = df.loc[train_index, 'num_label']
  #test_x = df.loc[test_index, 'tokens']
  #test_y = df.loc[test_index, 'num_label']


  #เปลี่ยน Dataframe เป็น dictionary เพื่อหลีกเลี่ยง pyarrow error
  train = df.loc[train_index, ["tokens","num_label"]].to_dict(orient="series")
  test = df.loc[test_index, ["tokens","num_label"]].to_dict(orient="series")
  
  #เก็บประโยค test
  list_input_test.append(test['tokens'])

  #เก็บ index ของ mask
  train_mask_index = mask_index[train_index]
  test_mask_index = mask_index[test_index]

  num_label = ['mask_1st', 'mask_2nd', 'mask_3rd', 'N']
  num_features= Features(
                {"tokens": Sequence(Value('string')),
                    "num_label": Sequence(feature=ClassLabel(names=num_label))
                })
  data = DatasetDict({'train': Dataset.from_dict(train,features=num_features), 
                 'test': Dataset.from_dict(test,features=num_features)})
  
  label_all_tokens = True
  tokenized_datasets = data.map(tokenize_and_align_labels, batched=True)


  # Finetuning
  target = "status"
  model_checkpoint = "airesearch/wangchanberta-base-att-spm-uncased"
  batch_size = 8 #8
  num_epoch = 17 #16
  condition = f"pretokenized_conn_status"
  limit_n_tokens = 200
  save="no"

  label_list = data['train'].features["num_label"].feature.names

  model = AutoModelForTokenClassification.from_pretrained(lm_name, num_labels=len(label_list))
  model_name = model_checkpoint.split("/")[-1]

  args = TrainingArguments(
      f"/content/drive/My Drive/discon model/'{model_name}-{num_epoch} epoch-{condition}",
      evaluation_strategy = "epoch",
      learning_rate=2e-5,
      per_device_train_batch_size=batch_size,
      num_train_epochs=num_epoch,
      save_strategy=save,
      #save_total_limit = 2, 
      #load_best_model_at_end=True,
      weight_decay=0.01,
      push_to_hub=False,
  )
  data_collator = DataCollatorForTokenClassification(tokenizer)
  metric = load_metric("seqeval")

  trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets['train'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)

  trainer.train()

  # Predict Test
  predictions, labels, _ = trainer.predict(tokenized_datasets['test'])
  predictions = np.argmax(predictions, axis=2)
  # Prediction: array([[3, 3, 3, ..., 0, 0, 0], [3, 3, 3, ..., 0, 0, 0], ... ])

  bert_prediction = [
    [(id, token, prediction) for (id, token, prediction) in zip(row_id, row_token,row_predictions) ]
    for row_id, row_token, row_predictions in zip(tokenized_datasets['test']['word_ids'],tokenized_datasets['test']['tokens_bert'],predictions)
  ]
  # bert_prediction: [[(None, '<s>', 3), (0, '▁มึง', 3), (1, '▁ช่วย', 3), (2, '▁', 3), (2, 'ให้', 3), (2, 'คําปรึกษา', 3),...] ...]

  gold_label = [
      [label for label in row_label ]
      for row_label in tokenized_datasets['test']['num_label']
  ]
  # gold label: [[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3], [3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3], ... ]


  wrap_predict = [] 
  for row in bert_prediction:
      predict_row=[]
      previous_word=None
      for item in row:
          if(item[0] is not None):
              #skip special token
              if(item[0] !=previous_word):
                  predict_row.append(item[-1])
                  previous_word = item[0]
      wrap_predict.append(predict_row)


  #from number to numper
  flat_true_predictions = flatten(wrap_predict)
  flat_true_labels = flatten(gold_label)

  # take only mask
  wrap_predict = [row[index] for row, index in zip(wrap_predict, test_mask_index)]
  gold_label = [row[index] for row, index in zip(gold_label, test_mask_index)]

  list_predict.append(wrap_predict)
  list_actual.append(gold_label)


  result = classification_report(flat_true_labels,flat_true_predictions,output_dict=True,labels=num_label)
  result_df=pd.DataFrame(result).transpose()
  print (result_df)

  0%|          | 0/1 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/1 [00:00<?, ?ba/s]

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertForTokenClassification: ['lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing CamembertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier

Downloading:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/airesearch/wangchanberta-base-att-spm-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/616a9e2dfc52e9d019b75d219ed800a27158ed299bd4fad91363110fe93dfce1.27c4f6581fbedf3d12e9fae96d4fbb8bc3064cd88ae545414e7cffc7c5bbc52f
Model config CamembertConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "camembert",
  "num_attention_head": 12,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.270165,0.000000,0.000000,0.000000,0.929204
2,No log,0.168525,0.157025,0.098958,0.121406,0.942655
3,No log,0.094806,0.528384,0.630208,0.574822,0.964602
4,No log,0.063835,0.680556,0.765625,0.720588,0.975929
5,No log,0.050096,0.803922,0.854167,0.828283,0.984071
6,No log,0.043724,0.859296,0.890625,0.874680,0.987257
7,No log,0.035923,0.859296,0.890625,0.874680,0.988319
8,No log,0.032464,0.897959,0.916667,0.907216,0.990088
9,No log,0.031246,0.907216,0.916667,0.911917,0.989381
10,No log,0.030876,0.892857,0.911458,0.902062,0.989381


The following columns in the evaluation set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: tokens, num_label, word_ids, tokens_bert.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: mask_2nd seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: N seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: mask_3rd seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: mask_1st seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))

              precision    recall  f1-score      support
mask_1st       0.848485  0.800000  0.823529    35.000000
mask_2nd       0.735294  0.862069  0.793651    29.000000
mask_3rd       0.878788  0.805556  0.840580    36.000000
N              1.000000  1.000000  1.000000  1646.000000
accuracy       0.989691  0.989691  0.989691     0.989691
macro avg      0.865642  0.866906  0.864440  1746.000000
weighted avg   0.990067  0.989691  0.989748  1746.000000


# df Gold vs Prediction

In [ ]:
def tokens_to_text(tokens):
  text = ''.join(tokens)
  text = re.sub('mask', '_____', text)
  return text

In [ ]:
predict_1 = list_predict[0]
predict_2 = list_predict[1]
predict_3 = list_predict[2]
actual_1 = list_actual[0]
actual_2 = list_actual[1]
actual_3 = list_actual[2]

input_test_1 = list_input_test[0]
input_test_2 = list_input_test[1]
input_test_3 = list_input_test[2]

In [ ]:
df_predict_vs_gold_1 = pd.DataFrame({'text':input_test_1, 'actual':actual_1, 'predict':predict_1})
df_predict_vs_gold_2 = pd.DataFrame({'text':input_test_2, 'actual':actual_2, 'predict':predict_2})
df_predict_vs_gold_3 = pd.DataFrame({'text':input_test_3, 'actual':actual_3, 'predict':predict_3})

df_predict_vs_gold_1['text'] = df_predict_vs_gold_1['text'].apply(tokens_to_text)
df_predict_vs_gold_2['text'] = df_predict_vs_gold_2['text'].apply(tokens_to_text)
df_predict_vs_gold_3['text'] = df_predict_vs_gold_3['text'].apply(tokens_to_text)

In [ ]:
df_predict_vs_gold_1['actual'] = df_predict_vs_gold_1['actual'].map({0:'<mask_1st>',1:'<mask_2nd>',2:'<mask_3rd>'})
df_predict_vs_gold_2['actual'] = df_predict_vs_gold_2['actual'].map({0:'<mask_1st>',1:'<mask_2nd>',2:'<mask_3rd>'})
df_predict_vs_gold_3['actual'] = df_predict_vs_gold_3['actual'].map({0:'<mask_1st>',1:'<mask_2nd>',2:'<mask_3rd>'})
df_predict_vs_gold_1['predict'] = df_predict_vs_gold_1['predict'].map({0:'<mask_1st>',1:'<mask_2nd>',2:'<mask_3rd>'})
df_predict_vs_gold_2['predict'] = df_predict_vs_gold_2['predict'].map({0:'<mask_1st>',1:'<mask_2nd>',2:'<mask_3rd>'})
df_predict_vs_gold_3['predict'] = df_predict_vs_gold_3['predict'].map({0:'<mask_1st>',1:'<mask_2nd>',2:'<mask_3rd>', 3:'<N>'})

In [ ]:
len(predict_3)

100

## Confusion Metrix

In [ ]:
def confusion_matrix(y_true, y_pred, n_class=3):
    matrix = np.zeros((n_class, n_class))
    if type(y_true[0]) != int:
        y_true = [int(re.search(r'.+(\d)', x).group(1))-1 for x in y_true]
        y_pred = [int(re.search(r'.+(\d)', x).group(1))-1 for x in y_pred]
    for i, j in zip(y_pred, y_true):
        matrix[i, j] += 1
    index_labels = [f'pred_{i}' for i in range(1, n_class+1)]
    column_labels = [f'actual_{i}' for i in range(1, n_class+1)]
    matrix = pd.DataFrame(matrix, index=index_labels, columns=column_labels, dtype=int)
    #matrix['sum'] = matrix.sum(axis=1)
    #matrix.loc['sum'] = matrix.sum(axis=0)
    return matrix

## Fold 1

In [ ]:
df_check_1 = df_predict_vs_gold_1[df_predict_vs_gold_1['actual'] != df_predict_vs_gold_1['predict']].sort_values('actual')

In [ ]:
confusion_matrix(df_predict_vs_gold_1['actual'].to_list(), df_predict_vs_gold_1['predict'].to_list())

,actual_1,actual_2,actual_3
pred_1,24,3,5
pred_2,2,23,8
pred_3,5,1,29


In [ ]:
df_check_1.sort_values('actual')

,text,actual,predict
150,ขอให้ผมกับคุณได้พบกัน หลังจากนั้นไม่นาน_____ก็...,<mask_1st>,<mask_3rd>
287,เขาก็อยากอยู่กับตัวเองไปตลอดชีวิตนะ แต่ตัวเองจ...,<mask_1st>,<mask_3rd>
33,ฉันเสียใจเพราะสามีไม่อนุญาตให้_____เลี้ยงแมว,<mask_1st>,<mask_3rd>
226,เดี๋ยวพวกกูกำลังตามไปนะ มึงจองที่ไว้ให้_____หน่อย,<mask_1st>,<mask_2nd>
76,พวกมึงไปกันก่อนเลย เพราะ_____ยังทำงานไม่เสร็จอ่ะ,<mask_1st>,<mask_2nd>
146,_____ไม่เจอกันกี่ปีๆ กลับมาเจอกันกี่ทีเราก็ยัง...,<mask_1st>,<mask_3rd>
141,ตอนกูไปลำปางญาติเยอะมาก ถ้า_____ติดโควิดก็คือต...,<mask_1st>,<mask_3rd>
184,เมื่อก่อนตอนเธออ้วน _____ดูแก่กว่าปัจจุบันอีก,<mask_2nd>,<mask_3rd>
162,วันนี้ฉันทำเธอร้องไห้ _____คงรู้สึกผิดหวังในสิ...,<mask_2nd>,<mask_1st>
10,ขอบคุณมากนะ เดี๋ยวเราพา_____ไปเลี้ยงข้าวเป็นกา...,<mask_2nd>,<mask_1st>


## Fold 2

In [ ]:
df_check_2 = df_predict_vs_gold_2[df_predict_vs_gold_2['actual'] != df_predict_vs_gold_2['predict']].sort_values('actual')

In [ ]:
confusion_matrix(df_predict_vs_gold_2['actual'].to_list(), df_predict_vs_gold_2['predict'].to_list())

,actual_1,actual_2,actual_3
pred_1,22,1,3
pred_2,2,31,5
pred_3,1,5,30


In [ ]:
df_check_2.sort_values('actual')

,text,actual,predict
216,เอ๊ะ..เรานัดกันหรือยังนะว่า_____จะจัดวันเกิดอิ...,<mask_1st>,<mask_2nd>
205,ช่วงนี้เจอพี่อาจจะเห็นว่า_____อ้วนหน่อยนะ 555,<mask_1st>,<mask_2nd>
248,ตอนผมเริ่มทำงานครั้งแรก พี่ในทีมชอบแซวผมว่า___...,<mask_1st>,<mask_3rd>
179,อยากให้พี่ๆในกลุ่มช่วยแนะนำร้านไส้อั่วที่_____...,<mask_2nd>,<mask_3rd>
73,ถ้า_____ขับไปเรื่อยๆ ก็จะเจอปั๊มน้ำมัน ไปรอเรา...,<mask_2nd>,<mask_1st>
149,มาแล้วหรอแม่คุณ กว่า_____จะโผล่หัวมาได้ ให้พวก...,<mask_2nd>,<mask_3rd>
51,ทุกงานจะดูง่าย เมื่อ_____ไม่ต้องทำเอง,<mask_2nd>,<mask_3rd>
21,เมื่อวานมึงกลับไปก่อนทำไม ทำไม_____ไม่ช่วยกูเลย,<mask_2nd>,<mask_3rd>
62,ผู้ชายอาจจะอยากรู้ว่าแกอยากมีลูกรึเปล่า เพราะถ...,<mask_2nd>,<mask_3rd>
175,พี่เป้บอกว่า_____ไม่อยากกินที่นั่นอ่ะ เราไปที่...,<mask_3rd>,<mask_1st>


## Fold 3

In [ ]:
df_check_3 = df_predict_vs_gold_3[df_predict_vs_gold_3['actual'] != df_predict_vs_gold_3['predict']].sort_values('actual')

In [ ]:
#df_predict_vs_gold_3['predict'].value_counts(dropna=False)

In [ ]:
confusion_matrix(df_predict_vs_gold_3['actual'].to_list(), df_predict_vs_gold_3['predict'].to_list())

,actual_1,actual_2,actual_3
pred_1,28,2,3
pred_2,5,25,4
pred_3,2,2,29


In [ ]:
df_check_3.sort_values('actual')

,text,actual,predict
276,ไว้ถ้า_____กลับบ้านแล้ว พวกเรานัดกันไปหาอาม่าก...,<mask_1st>,<mask_2nd>
7,_____ต้องขอโทษด้วยค่ะ น้องธุรการใหม่เขาไม่รู้ค่ะ,<mask_1st>,<mask_2nd>
215,วันนี้ เจอกันในงานบนเวทีใหญ่นะครับ ถ้าเพื่อนๆ ...,<mask_1st>,<mask_2nd>
163,กูชอบโน้ตก็จริง แต่_____คงเป็นแฟนกับมันไม่ได้ว่ะ,<mask_1st>,<mask_3rd>
237,พี่ส่งข้อมูลเข้าอีเมลของแต่ละคนให้แล้วนะครับ ถ...,<mask_1st>,<mask_2nd>
65,บ้านผมมีคนติดโควิดครับ แต่เมื่อเช้า_____ตรวจเอ...,<mask_1st>,<mask_3rd>
88,หมอคิดว่าพรุ่งนี้คนไข้น่าจะไม่ต้องมาหา_____แล้...,<mask_1st>,<mask_2nd>
54,คิดดีๆ อ่ะเรา คราวก่อน_____ก็ทำพลาดไปแล้วทีนึง,<mask_2nd>,<mask_1st>
11,เธอรอเราอีกแป๊ปนึงได้ป่ะ _____เข้าไปจอดรอในปั๊...,<mask_2nd>,<mask_1st>
98,เราจะขอยืมหมวกแกหน่อย วันก่อนเห็น_____ใส่แล้วส...,<mask_2nd>,<mask_3rd>
